<p><strong><font size="6">WALOUS</font></strong></p>

<p><strong><font size="6">Fusion LC classifications results</font></strong></p>

<p><strong><font size="6">Predict on tiles</font></strong></p>

This python code implement the method developed by ANAGEO (ULB). 

Code developped on Linux Mint 18.1 (Ubuntu Xenial 16.04) and GRASS GIS 7.3.svn (r71315).

# Table of Contents

<div id="toc"></div>

The following cell is a Javascript section of code for building the Jupyter notebook's table of content.

In [1]:
%%javascript
$.getScript('https://kmahelona.github.io/ipython_notebook_goodies/ipython_notebook_toc.js')

<IPython.core.display.Javascript object>

# Define working environment

**Import libraries**

In [2]:
# Import libraries needed for setting parameters of operating system 
import os
import sys
import csv
import tempfile
import subprocess

In [3]:
## Import multiprocessing and functools libraries
import multiprocessing
from multiprocessing import Pool
from functools import partial

** Add folder with SCR provided belong to this notebook**

In [4]:
# Add local module to the path
src = os.path.abspath('/export/miro/cbassine/fusion/RF_fusion/WALOUS-master/SRC')
if src not in sys.path:
    sys.path.append(src)

** Setup environment variables for TAIS DESKTOP (Linux Mint + GRASS Dev) **

Please edit the file in `../SRC/config.py`, containing the configuration parameters, according to your own computer setup. The following cell is used to run this file.



In [33]:
run /export/miro/cbassine/fusion/RF_fusion/WALOUS-master/SRC/config.py

In [34]:
print config_parameters

{'outputfolder_classifRF_tiles': '/export/miro/cbassine/fusion/RF_fusion/Results_RW/Classif_RF_TEST_arbustive/classif_tiles', 'outputfolder_classifRF': '/export/miro/cbassine/fusion/RF_fusion/Results_RW/Classif_RF_TEST_arbustive', 'outputfolder_Logfile': '/export/miro/cbassine/fusion/RF_fusion/Results_RW/Log_file', 'permanent_mapset': 'PERMANENT', 'GRASSBIN': '/usr/bin/grass76', 'locationepsg': '31370', 'outputfolder': '/export/miro/cbassine/fusion/RF_fusion/Results_RW', 'list_tiles': '/export/miro/cbassine/fusion/RF_fusion/Results_RW/list_tiles', 'pixel_classes_list': '/export/miro/cbassine/fusion/RF_fusion/Results_RW/pixel_classes_list', 'outputfolder_classifRF_csv': '/export/miro/cbassine/fusion/RF_fusion/Results_RW/Classif_RF_TEST_arbustive/classif_csv', 'gisdb': '/export/miro/cbassine/GRASSDATA', 'location': 'WALOUS_31370', 'outputfolder_classfeatures': '/export/miro/cbassine/fusion/RF_fusion/Results_RW/Classification_features', 'PYTHONLIB': '/usr/bin/python2.7', 'njobs': 6, 'outp

In [35]:
# Import functions that setup the environmental variables
import environ_variables as envi

In [36]:
# Set environmental variables
envi.setup_environmental_variables() 
# Display current environment variables of your computer
envi.print_environmental_variables()

LESS = -M -I -R 	
BASH_ENV = /usr/share/lmod/7.6.1/init/bash 	
CPU = x86_64 	
BASH_FUNC_module%% = () {  eval $($LMOD_CMD bash "$@") && eval $(${LMOD_SETTARG_CMD:-:} -s sh)
} 	
SSH_CONNECTION = 10.36.1.8 51492 10.36.2.4 22 	
SHELL = /bin/bash 	
XDG_DATA_DIRS = /usr/share 	
HISTSIZE = 1000 	
_ModuleTable003_ = aSIsfSx9LG1wYXRoQT17Ii91c3Ivc2hhcmUvbW9kdWxlcyIsIi91c3Ivc2hhcmUvbG1vZC9tb2R1bGVmaWxlcyIsfSxbInN5c3RlbUJhc2VNUEFUSCJdPSIvdXNyL3NoYXJlL2xtb2QvbW9kdWxlZmlsZXMiLH0= 	
LESS_ADVANCED_PREPROCESSOR = no 	
MANPATH = /usr/lib64/mpi/gcc/openmpi/share/man:/usr/local/man:/usr/share/man 	
JAVA_HOME = /usr/lib64/jvm/jre-1.8.0-openjdk 	
PROFILEREAD = true 	
XDG_RUNTIME_DIR = /run/user/49919 	
JPY_PARENT_PID = 32396 	
PYTHONPATH = /usr/lib64/grass76/etc/python:/export/apps/os150/gdal-2.2.4/gdal/lib64/python3.6/site-packages:/usr/lib64/grass76/etc/python:/usr/lib64/grass76/etc/python/grass:/usr/lib64/grass76/etc/python/grass/script:/usr/lib64/grass76/etc/python:/usr/lib64/grass76/etc/python/grass:/

** GRASS GIS Python libraries **

In [37]:
from grass_session import Session
# Import libraries needed to launch GRASS GIS in the jupyter notebook
import grass.script.setup as gsetup
# Import libraries needed to call GRASS using Python
import grass.script as gscript

**Import libraries**

In [38]:
# Import function that check and create folder
from mkdir import check_create_dir

In [39]:
# Import function that generate a random name in the GRASS GIS environement
from random_layer_name import random_layer_name

## Special functions

In [40]:
# Import function that check existance and create GRASS GIS database folder if needed
from grass_database import check_gisdb, check_location, check_mapset, working_mapset
# Import functions for processing time information
from processing_time import start_processing, print_processing_time

In [71]:
## Function that create a R script for the prediction on the current tile
def CreateRScript(tile_cat, r_ncores=2):
    import tempfile
    # Define names, input and output path... 
    classif_features = os.path.join(config_parameters['outputfolder_classfeatures'],"features_tile_%s.csv"%tile_cat).replace("\\", "/")
    print classif_features
    classif_result = os.path.join(config_parameters['outputfolder_classifRF_csv'],"RF_fusion_tile_%s.csv"%tile_cat).replace("\\", "/")
    print classif_result
    # Create and open new temporary R script
    r_commands = "%s.R"%tempfile.mktemp().replace("\\", "/")
    r_file = open(r_commands, 'w')

    # Create a list with all lines to write in the script
    r_instuction = []

    # Piece of R script for intallation of R package
    install_package = "if(!is.element('%s', installed.packages()[,1])){\n"
    install_package += "cat('\\n\\nInstalling %s package from CRAN\n')\n"
    install_package += "if(!file.exists(Sys.getenv('R_LIBS_USER'))){\n"
    install_package += "dir.create(Sys.getenv('R_LIBS_USER'), recursive=TRUE)\n"
    install_package += ".libPaths(Sys.getenv('R_LIBS_USER'))}\n"
    install_package += "chooseCRANmirror(ind=1)\n"
    install_package += "install.packages('%s', dependencies=TRUE)}"
    r_instuction.append(install_package % ('doParallel', 'doParallel', 'doParallel')) #Package for parallel processing in R
    r_instuction.append(install_package % ('caret', 'caret', 'caret')) #Package for machine learning in R
    r_instuction.append(install_package % ('e1071', 'e1071', 'e1071')) 
    r_instuction.append(install_package % ('randomForest', 'randomForest', 'randomForest')) #Package for Random Forest
    r_instuction.append(install_package % ('pROC', 'pROC', 'pROC')) 
    r_instuction.append(install_package % ('reticulate', 'reticulate', 'reticulate')) #Package for executing python script directly in Rscript

    # Piece of R script for loading libraries and dependencies
    lib_require = "library(%s)\n"
    lib_require += "require(%s)"
    r_instuction.append(lib_require % ('parallel', 'parallel')) 
    r_instuction.append(lib_require % ('doParallel', 'doParallel')) 
    r_instuction.append(lib_require % ('caret', 'caret')) 
    r_instuction.append(lib_require % ('randomForest', 'randomForest')) 
    r_instuction.append(lib_require % ('e1071', 'e1071')) 
    r_instuction.append(lib_require % ('pROC', 'pROC')) 
    r_instuction.append(lib_require % ('reticulate', 'reticulate')) 

    # Set parameter for enabling use of Python function for ERP computation inside of R
    r_instuction.append("# Set the path to the Python executable file")
    r_instuction.append("use_python('%s', required = T)"%config_parameters['PYTHONLIB'])
    r_instuction.append("source_python('/export/miro/cbassine/fusion/RF_fusion/WALOUS-master/SRC/EquivalentReferenceProbability.py')")
                        
    # Set number of cores to use
    r_instuction.append("# Set number of cores to use")
    r_instuction.append("usedcores=%s"%r_ncores)
    #r_instuction.append("usedcores=detectCores()-5")
    r_instuction.append("registerDoParallel(usedcores)")

    # Import object statistics as dataframe
    r_instuction.append("# Import object statistics as dataframe")
    r_instuction.append("features <- read.csv('%s', sep=',', header=TRUE, row.names=1)"%classif_features)
    
    # Load trained RF model
    r_instuction.append("# Load trained RF model")
    r_instuction.append("rfModel <- readRDS('%s')"%config_parameters['rf_trained_model'].replace("\\", "/"))
   
    # Predict class probability (SoftProb)
    r_instuction.append("# Predict probability of each class (SoftProb)")
    r_instuction.append("predicted_prob <- data.frame(predict(rfModel, features, type='prob'))")
    r_instuction.append("names(predicted_prob) <- substring(names(predicted_prob),2)")
    r_instuction.append("colnames(predicted_prob) <- paste('prob', colnames(predicted_prob), sep = '_')")
    # Export class prob to csv
    r_instuction.append("# Export class prob to csv")
    r_instuction.append("path_tmp_csv <- tempfile(pattern = 'file', tmpdir = tempdir(), fileext = '.csv')")
    r_instuction.append("write.csv(predicted_prob, file=path_tmp_csv, row.names=TRUE, quote=FALSE)")
                        
    # Compute ERP using dedicated Python function 
    r_instuction.append("# Compute ERP (Equivalent Reference Probability) using Python function")
    r_instuction.append("path_tmp_csv_ERP = ComputeERPfromCsv(path_tmp_csv, delimiter=',', erp_name='ERP', start_index=1L, stop_index=FALSE)")
    r_instuction.append("predicted_prob_ERP <- read.csv(path_tmp_csv_ERP, sep=',', header=TRUE, row.names=1)")
    
    # Predict class label (SoftMAX)
    r_instuction.append("# Predict label (SoftMAX)")    
    r_instuction.append("predicted_label <- data.frame(predict(rfModel, features))")
    r_instuction.append("names(predicted_label) <- 'rf_label'")
    
    # Merge all results in one final dataframe
    r_instuction.append("mnh_mean <- data.frame(features$mnh2018_mean/100)")
    r_instuction.append("names(mnh_mean) <- 'mnh_mean'")
    r_instuction.append("# Merge probabilities and label in final dataframe")
    r_instuction.append("rf_predictions <- data.frame(cat_=rownames(features), predicted_prob_ERP, predicted_label, mnh_mean)")
    r_instuction.append("rf_predictions$rf_label <- as.numeric(as.character(rf_predictions$rf_label))")
    r_instuction.append("rf_predictions[which(rf_predictions$rf_label==42 & rf_predictions$mnh_mean<=3),'rf_label']<-19")
    r_instuction.append("rf_predictions[which(rf_predictions$rf_label==41 & rf_predictions$mnh_mean<=3),'rf_label']<-18")

    # Export final result to csv
    r_instuction.append("# Export final result to csv")
    r_instuction.append("write.csv(rf_predictions, file='%s', row.names=FALSE, quote=FALSE)"%classif_result)

    # Write instructions to the R script file
    r_file.write("\n".join(r_instuction))
    r_file.close()
    
    # Return the path to the Rscript file
    return r_commands

In [42]:
def launch_mapset(mapset):
    #Declare empty list that will contain the messages to return
    return_message = []
    # Check if the location exists and create it if not, with the CRS defined by the epsg code 
    return_message.append(check_location(config_parameters["gisdb"],config_parameters['location'],config_parameters["locationepsg"]))
    # Check if mapset exists
    return_message.append(check_mapset(config_parameters["gisdb"],config_parameters['location'],mapset))
    # Change the current working GRASS GIS session mapset
    return_message.append(working_mapset(config_parameters["gisdb"],config_parameters['location'],mapset))
    # Return
    return return_message

In [43]:
def GetMapsetsAccess():
    #Declare empty string that will contain the messages to return
    return_message = ''
    try:
        # Add mapsets with input data to the GRASS GIS research path
        gscript.run_command('g.mapsets', mapset="OBIA", operation="add")
        return_message = "Access to other mapset added"
    except:
        return_message += "ERROR: Add access to other Mapsets failed. Please check for problem."
    return return_message

In [44]:
def DefineComputationRegion(tile_cat):
    #Declare empty string that will contain the messages to return
    return_message = ''
    try:
        return_message = "Working on tile '%s'\n"%tile_cat
        gscript.run_command('g.region', raster='segs_tile_%s'%tile_cat)
        # Print
        return_message += "--> Computational region and MASK defined"
    except:
        return_message += "ERROR: Setting of computional region failed for cutline '%s'. Please check for problem."%tile_cat
    return return_message

In [45]:
def ReclassWithPropsERPLabel(tile_cat):
    global list_rast_fusion
    list_rast_fusion = []
    #Declare empty string that will contain the messages to return
    return_message = ''
    try:
        # Read the header of the CSV file with results from R
        classif_result = os.path.join(config_parameters['outputfolder_classifRF_csv'],"RF_fusion_tile_%s.csv"%tile_cat)
        fin = open(classif_result)
        reader = csv.reader(fin)
        header = reader.next()
        
        # Recode or reclass segments according to R outputs (probabilities, ERP and label)
        for i,col in enumerate(header):
            if col == header[0]: #Skip first colunm with 'cat_'
                continue
            reader = csv.reader(open(classif_result))
            reader.next() #Skip the header
            #Only if current loop is related to the last columns (label in Integer)
            if col == header[-1]: 
                temprulecsv = tempfile.mktemp() # Define the csv output file name
                fout = open(temprulecsv, 'w')
                for in_row in reader:
                    fout.write("%s=%s"%(in_row[0],in_row[i]))
                    fout.write("\n")
                fout.write("*=NULL")
                fout.close()
                ## Reclass segments raster layer to keep only outliers segments, using the reclas_rule.csv file (create temporary raster)
                prediction_raster = 'Fusion_%s'%col
                list_rast_fusion.append(prediction_raster)
                gscript.run_command('g.region', overwrite=True, raster='segs_tile_%s'%tile_cat)
                gscript.run_command('r.reclass', overwrite=True, input='segs_tile_%s'%tile_cat, 
                                  output=prediction_raster, rules=temprulecsv)
                os.remove(temprulecsv)            
            
            #For all the other columns (probs and ERP in float)
            else: 
                temprulecsv = tempfile.mktemp() #Define the csv output file name
                fout = open(temprulecsv, 'w')
                for in_row in reader:
                    fout.write("%s=%s"%(in_row[0],float(in_row[i])*10000))  #Multiply by 10.000 because r.recode support only integers (will round the value of the rule)
                    fout.write("\n")
                fout.close()
                ## Reclass segments raster layer to keep only outliers segments, using the reclas_rule.csv file (create temporary raster)
                gscript.run_command('g.region', overwrite=True, raster='segs_tile_%s'%tile_cat)
                tmp_raster = random_layer_name()
                gscript.run_command('r.reclass', overwrite=True, input='segs_tile_%s'%tile_cat, 
                                  output=tmp_raster, rules=temprulecsv)
                prediction_raster = 'Fusion_%s'%col
                list_rast_fusion.append(prediction_raster)
                formula = "%s=%s/10000.0"%(prediction_raster,tmp_raster) #Divide by 10.000 to have the initial float value rounded at the 4th decimal (O.XXXX)
                gscript.mapcalc(formula, overwrite=True)
                gscript.run_command('g.remove', flags='f', type="raster", name=tmp_raster)
                os.remove(temprulecsv)
        # Print
        return_message += "--> Segmentation raster reclassed"
    except:
        return_message += "ERROR: Reclassification of segmentation raster failed for cutline '%s'. Please check for problem."%tile_cat
    return return_message

In [20]:
def Colorize(tile_cat):
    #Declare empty string that will contain the messages to return
    return_message = ''
    try:
        gscript.run_command('r.colors', map='Fusion_rf_label', rules=data['color_file'])
        gscript.run_command('r.colors', flags='e', map='Fusion_ERP', color='viridis')
        return_message += "--> Colors applied"
    except:
        return_message += "ERROR: Application of colors failed. Please check for problem."%tile_cat
    return return_message

In [46]:
"""
Export only the label of the RF classification (majority class)
"""
def ExportTiff(tile_cat):
    global list_rast_fusion
    #Declare empty string that will contain the messages to return
    return_message = ''
    try:
        export_path = os.path.join(config_parameters['outputfolder_classifRF_tiles'],"RF_fusion_tile_%s.tif"%tile_cat)
        # Export the group as a .tif file
        gscript.run_command('r.out.gdal', quiet=True, overwrite=True, flags='cm', input="Fusion_rf_label", output=export_path,
                            format='GTiff', createopt='COMPRESS=DEFLATE') #Flag c to not export colortable. Flag m to not export non-standard format of meta-data
        return_message += "--> Classified raster exported"
    except:
        return_message += "ERROR: Export of classified raster failed for cutline '%s'. Please check for problem."%tile_cat
    return return_message

In [47]:
def Clean(tile_cat):
    #Declare empty string that will contain the messages to return
    return_message = ''
    try:
        for layer in list_rast_fusion:
            gscript.run_command('g.remove', flags='f', type="raster", name=layer)
        return_message += "--> Mapset cleaned"
    except:
        return_message += "ERROR: during mapset cleaning. Please check for problem."
    return return_message

In [48]:
def GetPrediction(tile_cat):
    import subprocess
    start_tile = start_processing() 
#    print "Start processing on tile %s"%tile_cat
    #Declare empty list for saving output messages
    output_message = [] 
    # Create R script
    r_commands = CreateRScript(tile_cat, r_ncores=4)
    # Execute R script
    message = "Running R now. Following output is R output."
    output_message.append(message)
    print message
    try:
        subprocess.check_call(['Rscript', r_commands], stderr=subprocess.STDOUT, )
    except subprocess.CalledProcessError:
        message =  "There was an error in the execution of the R script.\nPlease check the R output."
        output_message.append(message)
        print message
    message =  "Finished running R."
    output_message.append(message)
    print message
    
    # Launch mapset
    message = launch_mapset(tile_cat)  
    [output_message.append(a) for a in message]
    print "\n".join(message)
    
    # Allow access to other mapset 
    message = GetMapsetsAccess()
    output_message.append(message)
    print message
    
    # Define computional region and mask
    message = DefineComputationRegion(tile_cat)
    output_message.append(message)
    print message    
    
    # Reclass with classification probabilities, ERP and labels
    message = ReclassWithPropsERPLabel(tile_cat)
    output_message.append(message)
#    print message
    
    # Colorize 
    #message = Colorize(tile_cat)
    #output_message.append(message)
#    print message

    # Export 
    message = ExportTiff(tile_cat)
    output_message.append(message)
#    print message
    
    # Clean 
    message = Clean(tile_cat)
    output_message.append(message)
#    print message

    #Print processing time
    message = print_processing_time(start_tile, "Prediction for tile '%s' achieved in "%tile_cat)
    output_message.append(message)
#    print message
    
    #Export Log file
    fout = open(os.path.join(config_parameters['outputfolder_Logfile'],"Log_Prediction_tile_%s.txt"%tile_cat),"w")
    [fout.writelines('%s\n'%content) for content in output_message]
    fout.close()

## Create new directories

In [49]:
# Check and create folder if needed
check_create_dir(config_parameters['outputfolder_classifRF'])
check_create_dir(config_parameters['outputfolder_classifRF_csv'])
check_create_dir(config_parameters['outputfolder_classifRF_tiles'])

The folder '/export/miro/cbassine/fusion/RF_fusion/Results_RW/Classif_RF_TEST_arbustive' has been created
The folder '/export/miro/cbassine/fusion/RF_fusion/Results_RW/Classif_RF_TEST_arbustive/classif_csv' has been created
The folder '/export/miro/cbassine/fusion/RF_fusion/Results_RW/Classif_RF_TEST_arbustive/classif_tiles' has been created


## Import list of tiles from file

In [50]:
import pickle

with open(config_parameters['list_tiles'], 'rb') as outputfile: 
    tile_list = pickle.load(outputfile) 

In [51]:
len(tile_list)

6372

## Create exemple of R script for prediction on first tile

In [52]:
output = os.path.join(config_parameters['outputfolder_classifRF'],"Fusion_walous_RF_Predict.R")

In [53]:
r_commands = CreateRScript(tile_list[0],r_ncores=4)

/export/miro/cbassine/fusion/RF_fusion/Results_RW/Classification_features/features_tile_1.csv
/export/miro/cbassine/fusion/RF_fusion/Results_RW/Classif_RF_TEST_arbustive/classif_csv/RF_fusion_tile_1.csv


In [54]:
import shutil
try:
    shutil.copy2(r_commands, output)
    print "Example of R script for prediction was saved on '%s'"%output
except:
    print "ERROR : An issue occurend when saving a copy of R script for prediction. Please check."

Example of R script for prediction was saved on '/export/miro/cbassine/fusion/RF_fusion/Results_RW/Classif_RF_TEST_arbustive/Fusion_walous_RF_Predict.R'


**-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-**

# Predict using random forest model (R)

In [55]:
# Set number of cores to use
ncores = 20

In [72]:
# Launch processes in parallel
start_parallel = start_processing()
GetPrediction(tile_list[204])
# Print
print_processing_time(start_parallel, "Computation (on %s cores) achieved in "%ncores)

/export/miro/cbassine/fusion/RF_fusion/Results_RW/Classification_features/features_tile_205.csv
/export/miro/cbassine/fusion/RF_fusion/Results_RW/Classif_RF_TEST_arbustive/classif_csv/RF_fusion_tile_205.csv
Running R now. Following output is R output.
Finished running R.
Location 'WALOUS_31370' already exist
'205' mapset already exists in location 'WALOUS_31370'
You are now working in mapset 'WALOUS_31370/205'
Access to other mapset added
Working on tile '205'
--> Computational region and MASK defined


'Computation (on 20 cores) achieved in 3 minutes and 28.3 seconds'

In [73]:
# Launch processes in parallel
start_parallel = start_processing()
p = Pool(ncores)
output = p.map(GetPrediction, tile_list[:])  # Launch the processes for as many items in the list (if function with a return, the returned results are ordered thanks to 'map' function)
p.close()
p.join()
# Print
print_processing_time(start_parallel, "Computation (on %s cores) achieved in "%ncores)

/export/miro/cbassine/fusion/RF_fusion/Results_RW/Classification_features/features_tile_161.csv
/export/miro/cbassine/fusion/RF_fusion/Results_RW/Classification_features/features_tile_81.csv
/export/miro/cbassine/fusion/RF_fusion/Results_RW/Classification_features/features_tile_1.csv
/export/miro/cbassine/fusion/RF_fusion/Results_RW/Classification_features/features_tile_561.csv
/export/miro/cbassine/fusion/RF_fusion/Results_RW/Classification_features/features_tile_401.csv
/export/miro/cbassine/fusion/RF_fusion/Results_RW/Classification_features/features_tile_321.csv
/export/miro/cbassine/fusion/RF_fusion/Results_RW/Classification_features/features_tile_241.csv
/export/miro/cbassine/fusion/RF_fusion/Results_RW/Classification_features/features_tile_801.csv
/export/miro/cbassine/fusion/RF_fusion/Results_RW/Classification_features/features_tile_641.csv
/export/miro/cbassine/fusion/RF_fusion/Results_RW/Classification_features/features_tile_481.csv
/export/miro/cbassine/fusion/RF_fusion/Resu

You are now working in mapset 'WALOUS_31370/1361'
Access to other mapset added
Working on tile '961'
--> Computational region and MASK defined
Working on tile '1361'
--> Computational region and MASK defined
Finished running R.
Location 'WALOUS_31370' already exist
'321' mapset already exists in location 'WALOUS_31370'
You are now working in mapset 'WALOUS_31370/321'
Access to other mapset added
Working on tile '321'
--> Computational region and MASK defined
Finished running R.
Location 'WALOUS_31370' already exist
'241' mapset already exists in location 'WALOUS_31370'
You are now working in mapset 'WALOUS_31370/241'
Access to other mapset added
/export/miro/cbassine/fusion/RF_fusion/Results_RW/Classification_features/features_tile_1202.csv
/export/miro/cbassine/fusion/RF_fusion/Results_RW/Classif_RF_TEST_arbustive/classif_csv/RF_fusion_tile_1202.csv
Running R now. Following output is R output.
Working on tile '241'
--> Computational region and MASK defined
/export/miro/cbassine/fusion

Running R now. Following output is R output.
Finished running R.
Location 'WALOUS_31370' already exist
'1282' mapset already exists in location 'WALOUS_31370'
You are now working in mapset 'WALOUS_31370/1282'
Access to other mapset added
Working on tile '1282'
--> Computational region and MASK defined
/export/miro/cbassine/fusion/RF_fusion/Results_RW/Classification_features/features_tile_242.csv
/export/miro/cbassine/fusion/RF_fusion/Results_RW/Classif_RF_TEST_arbustive/classif_csv/RF_fusion_tile_242.csv
Running R now. Following output is R output.
Finished running R.
Location 'WALOUS_31370' already exist
'962' mapset already exists in location 'WALOUS_31370'
You are now working in mapset 'WALOUS_31370/962'
Access to other mapset added
Working on tile '962'
--> Computational region and MASK defined
Finished running R.
Location 'WALOUS_31370' already exist
'802' mapset already exists in location 'WALOUS_31370'
You are now working in mapset 'WALOUS_31370/802'
Access to other mapset added

You are now working in mapset 'WALOUS_31370/562'
Access to other mapset added
Working on tile '562'
--> Computational region and MASK defined
Finished running R.
Location 'WALOUS_31370' already exist
'325' mapset already exists in location 'WALOUS_31370'
You are now working in mapset 'WALOUS_31370/325'
Access to other mapset added
Working on tile '325'
--> Computational region and MASK defined
Finished running R.
Location 'WALOUS_31370' already exist
'722' mapset already exists in location 'WALOUS_31370'
You are now working in mapset 'WALOUS_31370/722'
Access to other mapset added
Working on tile '722'
--> Computational region and MASK defined
Finished running R.
Location 'WALOUS_31370' already exist
'1042' mapset already exists in location 'WALOUS_31370'
You are now working in mapset 'WALOUS_31370/1042'
Access to other mapset added
Working on tile '1042'
--> Computational region and MASK defined
Finished running R.
Location 'WALOUS_31370' already exist
'243' mapset already exists in l

Running R now. Following output is R output.
/export/miro/cbassine/fusion/RF_fusion/Results_RW/Classification_features/features_tile_328.csv
/export/miro/cbassine/fusion/RF_fusion/Results_RW/Classif_RF_TEST_arbustive/classif_csv/RF_fusion_tile_328.csv
Running R now. Following output is R output.
Finished running R.
Location 'WALOUS_31370' already exist
'244' mapset already exists in location 'WALOUS_31370'
You are now working in mapset 'WALOUS_31370/244'
Access to other mapset added
Working on tile '244'
--> Computational region and MASK defined
Finished running R.
Location 'WALOUS_31370' already exist
'483' mapset already exists in location 'WALOUS_31370'
You are now working in mapset 'WALOUS_31370/483'
Access to other mapset added
Working on tile '483'
--> Computational region and MASK defined
Finished running R.
Location 'WALOUS_31370' already exist
'328' mapset already exists in location 'WALOUS_31370'
You are now working in mapset 'WALOUS_31370/328'
Access to other mapset added
Wo

...

Location 'WALOUS_31370' already exist
'6310' mapset already exists in location 'WALOUS_31370'
You are now working in mapset 'WALOUS_31370/6310'
Access to other mapset added
Working on tile '6310'
--> Computational region and MASK defined
Finished running R.
Location 'WALOUS_31370' already exist
'6233' mapset already exists in location 'WALOUS_31370'
You are now working in mapset 'WALOUS_31370/6233'
Access to other mapset added
Working on tile '6233'
--> Computational region and MASK defined
/export/miro/cbassine/fusion/RF_fusion/Results_RW/Classification_features/features_tile_6074.csv
/export/miro/cbassine/fusion/RF_fusion/Results_RW/Classif_RF_TEST_arbustive/classif_csv/RF_fusion_tile_6074.csv
Running R now. Following output is R output.
/export/miro/cbassine/fusion/RF_fusion/Results_RW/Classification_features/features_tile_6311.csv
/export/miro/cbassine/fusion/RF_fusion/Results_RW/Classif_RF_TEST_arbustive/classif_csv/RF_fusion_tile_6311.csv
Running R now. Following output is R outp

You are now working in mapset 'WALOUS_31370/5919'
Access to other mapset added
Working on tile '5919'
--> Computational region and MASK defined
/export/miro/cbassine/fusion/RF_fusion/Results_RW/Classification_features/features_tile_6313.csv
/export/miro/cbassine/fusion/RF_fusion/Results_RW/Classif_RF_TEST_arbustive/classif_csv/RF_fusion_tile_6313.csv
Running R now. Following output is R output.
/export/miro/cbassine/fusion/RF_fusion/Results_RW/Classification_features/features_tile_5822.csv
/export/miro/cbassine/fusion/RF_fusion/Results_RW/Classif_RF_TEST_arbustive/classif_csv/RF_fusion_tile_5822.csv
Running R now. Following output is R output.
Finished running R.
Location 'WALOUS_31370' already exist
'6313' mapset already exists in location 'WALOUS_31370'
You are now working in mapset 'WALOUS_31370/6313'
Access to other mapset added
Working on tile '6313'
--> Computational region and MASK defined
Finished running R.
Location 'WALOUS_31370' already exist
'6238' mapset already exists in 

You are now working in mapset 'WALOUS_31370/6079'
Access to other mapset added
Working on tile '6079'
--> Computational region and MASK defined
/export/miro/cbassine/fusion/RF_fusion/Results_RW/Classification_features/features_tile_5826.csv
/export/miro/cbassine/fusion/RF_fusion/Results_RW/Classif_RF_TEST_arbustive/classif_csv/RF_fusion_tile_5826.csv
Running R now. Following output is R output.
Finished running R.
Location 'WALOUS_31370' already exist
'5826' mapset already exists in location 'WALOUS_31370'
You are now working in mapset 'WALOUS_31370/5826'
Access to other mapset added
Working on tile '5826'
--> Computational region and MASK defined
Finished running R.
Location 'WALOUS_31370' already exist
'6316' mapset already exists in location 'WALOUS_31370'
You are now working in mapset 'WALOUS_31370/6316'
Access to other mapset added
Working on tile '6316'
--> Computational region and MASK defined
/export/miro/cbassine/fusion/RF_fusion/Results_RW/Classification_features/features_til

Running R now. Following output is R output.
Finished running R.
Location 'WALOUS_31370' already exist
'5833' mapset already exists in location 'WALOUS_31370'
You are now working in mapset 'WALOUS_31370/5833'
Access to other mapset added
Working on tile '5833'
--> Computational region and MASK defined
/export/miro/cbassine/fusion/RF_fusion/Results_RW/Classification_features/features_tile_5834.csv
/export/miro/cbassine/fusion/RF_fusion/Results_RW/Classif_RF_TEST_arbustive/classif_csv/RF_fusion_tile_5834.csv
Running R now. Following output is R output.
Finished running R.
Location 'WALOUS_31370' already exist
'5834' mapset already exists in location 'WALOUS_31370'
You are now working in mapset 'WALOUS_31370/5834'
Access to other mapset added
Working on tile '5834'
--> Computational region and MASK defined
/export/miro/cbassine/fusion/RF_fusion/Results_RW/Classification_features/features_tile_5835.csv
/export/miro/cbassine/fusion/RF_fusion/Results_RW/Classif_RF_TEST_arbustive/classif_csv/

'Computation (on 20 cores) achieved in 19 hours and 57 minutes and 24.1 seconds'

**Check log file for ERRORS**

In [36]:
# Get list of csv with classification feature of individual tiles
import glob
list_log = glob.glob(os.path.join(config_parameters['outputfolder_Logfile'],"Log_Prediction_tile_*.txt"))
print "%s log files in the folder"%len(list_log)

6372 log files in the folder


In [37]:
# Declare new counter
count = 0
# Declare new list that will contain list of tile with error
tile_error_list = []
# Loop on list of log file
for logfile in list_log:
    got_error = False
    tile_num = os.path.splitext(os.path.basename(logfile))[0].split("_")[-1]
    fin = open(logfile, 'r')
    for row in fin:
        if row[:5] == "ERROR":  # If at least one line have error message, the whole file will be counted as 1 error
            got_error = True
    if got_error:    
        count += 1
        tile_error_list.append(tile_num)  # Add tile number to the list
# Print
print "%s tile(s) faced an ERROR during the processing.\n"%count

# Update tile list with only tiles that have ERROR in log 
print "\n".join(["Error on tile %s"%(a) for a in tile_error_list])

30 tile(s) faced an ERROR during the processing.

Error on tile 1368
Error on tile 579
Error on tile 130
Error on tile 131
Error on tile 132
Error on tile 133
Error on tile 134
Error on tile 135
Error on tile 136
Error on tile 137
Error on tile 138
Error on tile 139
Error on tile 140
Error on tile 141
Error on tile 142
Error on tile 143
Error on tile 144
Error on tile 145
Error on tile 146
Error on tile 147
Error on tile 148
Error on tile 2154
Error on tile 3865
Error on tile 3251
Error on tile 3569
Error on tile 3897
Error on tile 3748
Error on tile 5683
Error on tile 5200
Error on tile 6232


**-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-**

# Create VRT with all tiles

## On the computer

### All bands

In [38]:
# Path for .vrt
export_path = os.path.join(config_parameters['outputfolder_classifRF_tiles'],"RF_fusion_walous.vrt")

In [39]:
# Get a file with list of tiles
tiles_path = glob.glob(os.path.join(config_parameters['outputfolder_classifRF_tiles'],"RF_fusion_tile_*.tif"))
tmp_file = "%s_VRT_LIST.txt"%tempfile.mktemp()
with open(tmp_file, 'w') as f:
    f.writelines("\n".join(tiles_path))
f.close()

In [40]:
# Create VRT
try:
    list_argument = ['gdalbuildvrt', '-resolution', 'average', '-input_file_list', tmp_file, export_path]
    subprocess.check_call(list_argument, stderr=subprocess.STDOUT, )
except subprocess.CalledProcessError:
    print "There was an error in the creation of VRT. Please check."

### Band 11 only (label)

In [36]:
# Path for .vrt
export_path = os.path.join(config_parameters['outputfolder_classifRF_tiles'],"RF_fusion_walous_label.vrt")

In [37]:
# Get a file with list of tiles
tiles_path = glob.glob(os.path.join(config_parameters['outputfolder_classifRF_tiles'],"RF_fusion_tile_*.tif"))
tmp_file = "%s_VRT_LIST.txt"%tempfile.mktemp()
with open(tmp_file, 'w') as f:
    f.writelines("\n".join(tiles_path))
f.close()

In [38]:
# Create VRT
try:
    list_argument = ['gdalbuildvrt', '-b', '11', '-resolution', 'average', '-input_file_list', tmp_file, export_path]
    subprocess.check_call(list_argument, stderr=subprocess.STDOUT, )
except subprocess.CalledProcessError:
    print "There was an error in the creation of VRT. Please check."